# <center> 4.2 - LightGBM</center> 

<center><img src="https://lightgbm.readthedocs.io/en/latest/_images/LightGBM_logo_black_text.svg" width="400" height="100"></center>

LGBM (Light Gradient Boosted Machine) - была описана Голинь К., и соавт. в статье 2017 года [«LightGBM: A Highly Efficient Gradient Boosting Decision Tree»](https://papers.nips.cc/paper/6907-lightgbm-a-highly-efficient-gradient-boosting-decision-tree). Реализация вводит две ключевые идеи: GOSS и EFB.<br>
<br>
С помощью GOSS (Градиентная односторонняя выборка) мы исключаем значительную часть экземпляров данных с небольшими градиентами и используем оставшиеся экземпляры для оценки прироста информации. Было доказано, что, поскольку экземпляры данных с большими градиентами играют более важную роль, GOSS может получить довольно точную оценку с гораздо меньшим размером данных. <br>
<br>
Exclusive Feature Bundling (объединение взаимоисключающих признаков), или EFB, — это подход объединения разреженных (в основном нулевых) взаимоисключающих признаков, таких как категориальные переменные, закодированные onehot  кодированием. Это, по сути, тип автоматического подбора признаков.
>… мы пакетируем взаимоисключающие признаки (то есть они редко принимают ненулевые значения одновременно), чтобы уменьшить количество признаков.
<br>
<br>
>Мы называем нашу новую реализацию GBDT с помощью GOSS и EFB LightGBM. Наши эксперименты на нескольких общедоступных наборах данных показывают, что LightGBM ускоряет процесс обучения обычного GBDT более чем в 20 раз, достигая почти такой же точности.

In [77]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, mean_squared_error
from sklearn.preprocessing import LabelEncoder

In [89]:
car_info = pd.read_csv('../data/car_info.csv')   # car_info - информация про машины с таргетом
 
rides_info = pd.read_csv('../data/rides_info.csv') # rides_info - информация про поездки

In [90]:
rides_info = rides_info.merge(car_info, on = 'car_id', how = 'left')

In [4]:
drop_cols = ['user_id', 'car_id', 'ride_id', 'ride_date']
cat_cols = ['car_type', 'fuel_type', 'model']

In [5]:
# закодируем категориальные фичи в one hot encoding вектора
rides_info = pd.get_dummies(rides_info, columns=cat_cols)

In [6]:
# заполним пропущенные значения медианным значением по столбцу
rides_info.fillna(rides_info.median(), inplace=True)

In [ ]:
v

Особенность LightGBM — в том, что он поддерживает ряд алгоритмов бустинга, называемых типами бустинга. Тип бустинга указывается с помощью аргумента `boosting_type`.<br> Возможные значения:
* `gbdt` - градиеный бустинг над решающими деревьями (используется по умолчанию);
* `dart` - понятие отсева (dropout) вводится в MART, получаем DART (как [droput](https://ru.wikipedia.org/wiki/%D0%98%D1%81%D0%BA%D0%BB%D1%8E%D1%87%D0%B5%D0%BD%D0%B8%D0%B5_(%D0%BD%D0%B5%D0%B9%D1%80%D0%BE%D0%BD%D0%BD%D1%8B%D0%B5_%D1%81%D0%B5%D1%82%D0%B8)) слой в нейронках);
* `goss` - односторонняя выборка на основе градиента (GOSS).

